In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import r2_score, mean_squared_error

# Load the dataset
file_path = 'environmental_factors.csv'
data = pd.read_csv(file_path)

# Convert `date` to datetime and set it as the index
data['date'] = pd.to_datetime(data['date'], format='%d-%m-%Y %H:%M')
data.set_index('date', inplace=True)

# Replace invalid values (e.g., -9999) with NaN for handling
data.replace(-9999, np.nan, inplace=True)

# Impute Missing Values using Interpolation
data['wv'] = data['wv'].interpolate(method='time')
data['max. PAR'] = data['max. PAR'].interpolate(method='time')

# Outlier Detection and Handling (Capping using IQR)
def cap_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    df[column] = np.clip(df[column], lower_bound, upper_bound)

columns_to_cap = ['T', 'rh', 'rain', 'wv', 'max. PAR']
for col in columns_to_cap:
    cap_outliers(data, col)

# Feature Engineering (Adding Time-Based Features)
data['hour'] = data.index.hour
data['day'] = data.index.day
data['month'] = data.index.month
data['weekofyear'] = data.index.isocalendar().week

# Rolling averages for trend capture
data['T_roll_mean'] = data['T'].rolling(window=3).mean()
data['rh_roll_mean'] = data['rh'].rolling(window=3).mean()

# Drop rows with NaN introduced by rolling averages
data_cleaned = data.dropna()

# Split the data into features (X) and target variables (y)
X = data_cleaned.drop(['T', 'rh'], axis=1)
y_T = data_cleaned['T']  # Target variable for Temperature
y_rh = data_cleaned['rh']  # Target variable for Relative Humidity

# Split into training and test sets
X_train, X_test, y_train_T, y_test_T = train_test_split(X, y_T, test_size=0.2, random_state=42, shuffle=False)
X_train, X_test, y_train_rh, y_test_rh = train_test_split(X, y_rh, test_size=0.2, random_state=42, shuffle=False)

In [4]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error

# Initialize and train the model
model_lr = LinearRegression()
model_lr.fit(X_train, y_train_T)

# Predict Temperature (T)
y_pred_T_lr = model_lr.predict(X_test)
r2_T_lr = r2_score(y_test_T, y_pred_T_lr)
mse_T_lr = mean_squared_error(y_test_T, y_pred_T_lr)

# Predict Relative Humidity (rh)
model_lr.fit(X_train, y_train_rh)
y_pred_rh_lr = model_lr.predict(X_test)
r2_rh_lr = r2_score(y_test_rh, y_pred_rh_lr)
mse_rh_lr = mean_squared_error(y_test_rh, y_pred_rh_lr)

# Print Results
print("Linear Regression Results:")
print(f"Temperature R2: {r2_T_lr}")
print(f"Temperature MSE: {mse_T_lr}")
print(f"Humidity R2: {r2_rh_lr}")
print(f"Humidity MSE: {mse_rh_lr}")


Linear Regression Results:
Temperature R2: 0.9999955948614538
Temperature MSE: 0.00010959609728820012
Humidity R2: 0.9977355494294899
Humidity MSE: 0.3032907669227803


In [5]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score, mean_squared_error

# Initialize and train the model
model_dt = DecisionTreeRegressor()
model_dt.fit(X_train, y_train_T)

# Predict Temperature (T)
y_pred_T_dt = model_dt.predict(X_test)
r2_T_dt = r2_score(y_test_T, y_pred_T_dt)
mse_T_dt = mean_squared_error(y_test_T, y_pred_T_dt)

# Predict Relative Humidity (rh)
model_dt.fit(X_train, y_train_rh)
y_pred_rh_dt = model_dt.predict(X_test)
r2_rh_dt = r2_score(y_test_rh, y_pred_rh_dt)
mse_rh_dt = mean_squared_error(y_test_rh, y_pred_rh_dt)

# Print Results
print("Decision Tree Regressor Results:")
print(f"Temperature R2: {r2_T_dt}")
print(f"Temperature MSE: {mse_T_dt}")
print(f"Humidity R2: {r2_rh_dt}")
print(f"Humidity MSE: {mse_rh_dt}")


Decision Tree Regressor Results:
Temperature R2: 0.9999975827736874
Temperature MSE: 6.013853306765323e-05
Humidity R2: 0.9869724474369388
Humidity MSE: 1.7448543410190722


In [6]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error

# Initialize and train the model
model_rf = RandomForestRegressor()
model_rf.fit(X_train, y_train_T)

# Predict Temperature (T)
y_pred_T_rf = model_rf.predict(X_test)
r2_T_rf = r2_score(y_test_T, y_pred_T_rf)
mse_T_rf = mean_squared_error(y_test_T, y_pred_T_rf)

# Predict Relative Humidity (rh)
model_rf.fit(X_train, y_train_rh)
y_pred_rh_rf = model_rf.predict(X_test)
r2_rh_rf = r2_score(y_test_rh, y_pred_rh_rf)
mse_rh_rf = mean_squared_error(y_test_rh, y_pred_rh_rf)

# Print Results
print("Random Forest Regressor Results:")
print(f"Temperature R2: {r2_T_rf}")
print(f"Temperature MSE: {mse_T_rf}")
print(f"Humidity R2: {r2_rh_rf}")
print(f"Humidity MSE: {mse_rh_rf}")


Random Forest Regressor Results:
Temperature R2: 0.9999986202099593
Temperature MSE: 3.4328001707941535e-05
Humidity R2: 0.9954754806330234
Humidity MSE: 0.6059946578821528


In [7]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import r2_score, mean_squared_error

# Initialize and train the model
model_gb = GradientBoostingRegressor()
model_gb.fit(X_train, y_train_T)

# Predict Temperature (T)
y_pred_T_gb = model_gb.predict(X_test)
r2_T_gb = r2_score(y_test_T, y_pred_T_gb)
mse_T_gb = mean_squared_error(y_test_T, y_pred_T_gb)

# Predict Relative Humidity (rh)
model_gb.fit(X_train, y_train_rh)
y_pred_rh_gb = model_gb.predict(X_test)
r2_rh_gb = r2_score(y_test_rh, y_pred_rh_gb)
mse_rh_gb = mean_squared_error(y_test_rh, y_pred_rh_gb)

# Print Results
print("Gradient Boosting Regressor Results:")
print(f"Temperature R2: {r2_T_gb}")
print(f"Temperature MSE: {mse_T_gb}")
print(f"Humidity R2: {r2_rh_gb}")
print(f"Humidity MSE: {mse_rh_gb}")


Gradient Boosting Regressor Results:
Temperature R2: 0.9997923087196665
Temperature MSE: 0.005167182263863036
Humidity R2: 0.9955395378910785
Humidity MSE: 0.5974151043358977
